In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import time

In [ ]:
csv_file_path = 'C:\Data_Sets\Ml_datasets\predictive_analysis\Stock_Anomalies\data_ano.csv'  
data = pd.read_csv(csv_file_path, parse_dates=True, index_col='timestamp')  
print(data.head())

In [ ]:

def add_features(df):
    df['change_in_price'] = df['close'].pct_change()  
    df['change_in_volume'] = df['volume'].pct_change()  
    df['moving_avg_30'] = df['close'].rolling(window=30).mean()  
    df['volatility'] = df['close'].rolling(window=30).std() 
    df = df.dropna()
    return df

data_with_features = add_features(data)
print(data_with_features.head())


In [ ]:
def preprocess_data(df):
    features = ['change_in_price', 'change_in_volume', 'moving_avg_30', 'volatility']
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df[features])
    return df_scaled, df
X, original_df = preprocess_data(data_with_features)

In [ ]:
model = IsolationForest(contamination=0.01, random_state=42)
model.fit(X)
original_df['anomaly'] = model.predict(X)
print(original_df[['close', 'anomaly']].tail())
anomalies = original_df[original_df['anomaly'] == -1]
print("Anomalies detected:\n", anomalies)


In [ ]:
plt.figure(figsize=(14,7))
plt.plot(original_df.index, original_df['close'], label='Stock Price', color='blue')
plt.scatter(anomalies.index, anomalies['close'], color='red', label='Anomalies', marker='x')
plt.title('Stock Price with Anomalies')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


In [ ]:
for i in range(len(data_with_features)):
    current_data = data_with_features.iloc[[i]]
    X_new, _ = preprocess_data(current_data)
    prediction = model.predict(X_new)
    
    if prediction == -1:
        print(f"Anomaly detected at {current_data.index[0]}: {current_data['close'].values[0]}")
    
    time.sleep(1) 
